In [194]:
from ACLScheduleTest.Simulate import Simulator
from ACLScheduleTest.Analysis import Analyst
from ACLScheduleTest.data import RandomDsk
import numpy as np
from time import time

rd = RandomDsk(
    mute=True, node_num=100,
    st_node_num=10,
    min_time=1, time_lv=0, time_rv=0, min_time_step=0,
    avg_mem_peek=1, mem_pv=0, mem_sv=1
)
dsk = rd.build()
sim = Simulator()

In [195]:
num_workers, num_nodes = 10, 100

information = np.ones((num_nodes, num_workers))
def topoOrder(workers, cache, av_mem, factor_node_dict, calculated_node_dict, dsk, global_dict):
    topoQueue = []
    matrix = np.zeros((num_nodes, num_nodes))
    nodeList = list(dsk.keys())
    for idx, node in enumerate(nodeList):
        for n in dsk[node][1:]:
            if n in nodeList:
                id = nodeList.index(n)
                matrix[idx, id] = 1 # idx depends on id
    while len(topoQueue) < len(nodeList):
        for idx, node in enumerate(nodeList):
            if node in topoQueue:
                continue
            if matrix[idx].sum() == 0:
                topoQueue.append(node)
                matrix[:, idx] = 0
    return topoQueue


## Ant as Agent

In [196]:
current_worker_id = 0
iter = 0
from schedule_func_backup import schedule_out_func
def search():
    global current_worker_id, iter
    current_worker_id, iter = 0, 0
    worker_schedules = []
    for idx in range(num_workers):
        worker_schedule = []
        worker_schedules.append(worker_schedule)
    for node_id in range(num_nodes):
        prob = information[node_id] / information[node_id].sum()
        select_worker = np.random.choice(range(num_workers), p=prob)
        worker_schedules[select_worker].append(node_id)
    print(worker_schedules)
    def search_n_func(workers, cache, av_mem, factor_node_dict, calculated_node_dict, dsk, global_dict):
        global current_worker_id, iter
        ori_id = current_worker_id
        if av_mem <= 10:
            print(f'av_mem: {av_mem}')
        if av_mem == 1:
            return None
        if len(worker_schedules[ori_id])==0:
            return None
        node_id = worker_schedules[ori_id][0]
        node = f'node{node_id}'
        current_worker_id += 1
        if current_worker_id >= num_workers:
            current_worker_id = 0
            iter += 1
            print(f'\niter {iter}')
        #print(current_worker_id)
        nodeList = set(dsk.keys())
        print(node)

        pre_nodes = [arg for arg in dsk[node][1:] if arg in nodeList]
        print(pre_nodes)
        for arg in pre_nodes:
            if arg in nodeList and arg not in cache.keys():
                print('None\n', '-'*100)
                return None
        worker_schedules[ori_id].remove(node_id)
        print(node, '\n', '-'*100)
        return node
    result = sim.simulate(search_n_func, schedule_out_func, mem_bound=50, worker_bound=10, rd=rd)
    print(result)

## Train

In [197]:
def train(num_epochs, num_agent):
    for epoch in range(num_epochs):
        for agent_idx in range(num_agent):
            search()
train(1,1)

[[7, 19, 52, 58, 68, 79, 81, 83], [4, 25, 35, 41, 45, 46, 53, 92, 95, 97, 98], [5, 6, 34, 36, 57, 62, 65, 73, 87, 90], [2, 14, 37, 63, 66, 67, 88, 91, 94], [8, 9, 12, 18, 21, 22, 24, 30, 33, 39, 43, 44, 60, 82], [10, 15, 17, 23, 40, 49, 51, 56, 71, 76], [11, 29, 38, 55, 61, 70, 74, 77], [3, 13, 20, 27, 32, 42, 48, 50, 72, 75, 89, 93, 99], [1, 16, 31, 84, 96], [0, 26, 28, 47, 54, 59, 64, 69, 78, 80, 85, 86]]
node7
[]
node7 
 ----------------------------------------------------------------------------------------------------
node4
[]
node4 
 ----------------------------------------------------------------------------------------------------
node5
[]
node5 
 ----------------------------------------------------------------------------------------------------
node2
[]
node2 
 ----------------------------------------------------------------------------------------------------
node8
[]
node8 
 ----------------------------------------------------------------------------------------------------

## schedule_in_func 

## schedule_out_func